In [ ]:
# default_exp clone_counters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from functools import partial
from glob import glob

import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from skimage import measure

from py_clone_detective.utils import (
    add_scale_regionprops_table_area_measurements,
    calculate_corresponding_labels,
    calculate_overlap,
    check_channels_input_suitable_and_return_channels,
    determine_labels_across_other_images_using_centroids,
    extend_region_properties_list,
    get_all_labeled_clones_unmerged_and_merged,
    img_path_to_xarr,
    last2dims,
    lazy_props,
    reorder_df_to_put_ch_info_first,
    update_1st_coord_and_dim_of_xarr,
)

# CloneCounter Classes

## Parent Class

In [ ]:
# export
class CloneCounter:
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        self.exp_name, self.img_name_regex, self.pixel_size = (
            exp_name,
            img_name_regex,
            pixel_size,
        )

    def add_images(self, **channel_path_globs):
        return img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="img_channels",
            **channel_path_globs,
        )

    def add_segmentations(self, **channel_path_globs):
        segmentations = img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="seg_channels",
            **channel_path_globs,
        )
        segmentations.data = segmentations.data.map_blocks(
            last2dims(partial(measure.label)), dtype=np.uint16
        )
        return segmentations

    def combine_C0_overlaps_and_measurements(self):
        ov_df = (
            self.results_overlaps.pivot(
                index=["img_name", "C0_labels"],
                columns=["colocalisation_ch"],
                values="is_in_label",
            )
            .query("C0_labels != 0")
            .copy()
        )
        sk_df = self.results_measurements.query("seg_channel == 'C0'").set_index(
            ["segmentation_img_name", "label"]
        )
        sk_df.index.rename(["img_name", "C0_labels"], inplace=True)
        return pd.merge(ov_df, sk_df, left_index=True, right_index=True)

    def determine_seg_img_channel_pairs(
        self, seg_channels: list = None, img_channels: list = None
    ):
        seg_channels = check_channels_input_suitable_and_return_channels(
            channels=seg_channels,
            available_channels=self.image_data.seg_channels.values.tolist(),
        )

        img_channels = check_channels_input_suitable_and_return_channels(
            channels=img_channels,
            available_channels=self.image_data.img_channels.values.tolist(),
        )

        seg_img_channel_pairs = pd.DataFrame()
        seg_img_channel_pairs["image_channel"] = pd.Series(img_channels)
        seg_img_channel_pairs["segmentation_channel"] = pd.Series(seg_channels)
        self.seg_img_channel_pairs = seg_img_channel_pairs.fillna(method="ffill")[
            ["segmentation_channel", "image_channel"]
        ]

    def make_measurements(
        self,
        seg_channels: list = None,
        img_channels: list = None,
        extra_properties: list = None,
        **kwargs,
    ):

        self.determine_seg_img_channel_pairs(seg_channels, img_channels)

        properties = extend_region_properties_list(extra_properties)

        results = list()
        for _, seg_ch, img_ch in self.seg_img_channel_pairs.itertuples():
            for seg, img in zip(
                self.image_data["segmentations"].loc[seg_ch],
                self.image_data["images"].loc[img_ch],
            ):
                results.append(
                    lazy_props(
                        seg.data,
                        img.data,
                        seg.seg_channels.item(),
                        img.img_channels.item(),
                        seg.img_name.item(),
                        img.img_name.item(),
                        properties,
                        **kwargs,
                    )
                )

        df = dd.from_delayed(results).compute()
        df = add_scale_regionprops_table_area_measurements(df, self.pixel_size)
        self.results_measurements = reorder_df_to_put_ch_info_first(df)

    def _determine_max_seg_label_levels(self, seg_channel):
        """Determines in self.max_seg_label_levels is unassigned and create empty.
        Then adds `seg_channel` as new key and corresponding max segmentation label levels
        as value"""

        try:
            self.max_seg_label_levels.setdefault(seg_channel, []).append(
                (
                    self.image_data["segmentations"]
                    .loc[seg_channel]
                    .data.map_blocks(
                        lambda x: np.unique(x).shape[0],
                        drop_axis=(1, 2),
                        dtype=np.uint16,
                    )
                    .compute()
                    .max()
                )
            )
            self.max_seg_label_levels[seg_channel] = self.max_seg_label_levels[
                seg_channel
            ][0]
        except AttributeError:
            self.max_seg_label_levels = dict()
            self._determine_max_seg_label_levels(seg_channel)

    def _create_df_from_arr(self, arr):
        return (
            xr.DataArray(
                np.moveaxis(arr, 1, 0),
                coords=(
                    self.image_data["segmentations"].coords["seg_channels"][1:],
                    self.image_data["segmentations"].coords["img_name"],
                    np.arange(self.max_seg_label_levels["C0"]),
                ),
                dims=("colocalisation_ch", "img_name", "C0_labels",),
            )
            .to_dataframe("is_in_label")
            .reset_index()
            .dropna()
        )

    def measure_overlap(self):
        self._determine_max_seg_label_levels("C0")
        arr = (
            self.image_data["segmentations"]
            .data.map_blocks(
                calculate_overlap,
                drop_axis=[0],
                dtype=np.float64,
                num_of_segs=self.image_data["segmentations"].shape[0],
                preallocate_value=self.max_seg_label_levels["C0"],
            )
            .compute()
        )

        df = self._create_df_from_arr(arr)
        df["is_in_label"] = df["is_in_label"].astype(np.uint16)
        self.results_overlaps = df[
            ["img_name", "C0_labels", "colocalisation_ch", "is_in_label"]
        ]

    def clones_to_keep_as_dict(self, query_for_pd: str):
        return (
            self.results_measurements.query(query_for_pd)
            .groupby("intensity_img_name")
            .agg({"label": lambda x: list(x)})["label"]
            .to_dict()
        )

    def get_centroids_list(self, tot_seg_ch: str = "C0"):
        df = self.results_measurements.query("intensity_img_channel == @tot_seg_ch")
        centroids_list = list()
        for img_name in df["intensity_img_name"].unique():
            centroids_list.append(
                (
                    df.query("intensity_img_name == @img_name")
                    .loc[:, ["centroid-0", "centroid-1"]]
                    .values.astype(int)
                )
            )
        return centroids_list

    def add_clones_and_neighbouring_labels(
        self,
        tot_seg_ch: str = "C0",
        query_for_pd: str = 'intensity_img_channel == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
    ):
        clone_coords, clone_dims = update_1st_coord_and_dim_of_xarr(
            self.image_data["images"],
            new_coord=[
                f"{tot_seg_ch}",
                f"{tot_seg_ch}_extended",
                f"{tot_seg_ch}_inside_clones",
                f"{tot_seg_ch}_outside_clones",
                f"merged_clones",
                f"{tot_seg_ch}_neighbour_counts",
                f"{tot_seg_ch}_inside_clones_neighbour_counts",
                f"{tot_seg_ch}_outside_clones_neighbour_counts",
            ],
            new_dim="extended_labels_neighbour_counts",
        )

        clones_to_keep = self.clones_to_keep_as_dict(query_for_pd)

        new_label_imgs = get_all_labeled_clones_unmerged_and_merged(
            self.image_data["segmentations"].loc[tot_seg_ch], clones_to_keep
        )

        self.image_data[f"{name_for_query}_clones_and_neighbour_counts"] = xr.DataArray(
            data=new_label_imgs,
            coords=clone_coords,
            dims=clone_dims,
            attrs={f"{tot_seg_ch}_labels_kept_query": query_for_pd},
        )

## CloneCounter subclasses

In [ ]:
# export
class LazyCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        )

    def add_segmentations(self, **channel_path_globs):
        self.image_data["segmentations"] = super().add_segmentations(
            **channel_path_globs
        )

In [ ]:
# export
class PersistentCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        ).persist()

    def add_segmentations(self, **channel_path_globs):
        self.image_data["segmentations"] = (
            super().add_segmentations(**channel_path_globs).persist()
        )

In [ ]:
# hide
from dask.distributed import Client

c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:54463,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:54479,Total threads: 2
Dashboard: http://127.0.0.1:54480/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:54467,


## Example using LazyCloneCounter with measure_overlap

In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

foo.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
foo.make_measurements(extra_properties=["convex_area"],)
foo.measure_overlap()
foo.combine_C0_overlaps_and_measurements()

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


## Example using LazyCloneCounter with add_clones_and_neighbouring_labels

In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

foo.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
)
foo.make_measurements(extra_properties=["convex_area"],)
foo.add_clones_and_neighbouring_labels(
    tot_seg_ch="C0",
    query_for_pd='intensity_img_channel == "C1" & mean_intensity > 1000',
    name_for_query="filt_C1_intensity",
)

In [ ]:
#hide
centroids_list = foo.get_centroids_list("C0")

In [ ]:
#hide
labels = foo.image_data["filt_C1_intensity_clones_and_neighbour_counts"]

In [ ]:
#hide
foo._determine_max_seg_label_levels("C0")

In [ ]:
#hide
bar = test_func(
    labels.data, centroids_list, 8, foo.max_seg_label_levels["C0"]
).compute()

In [ ]:
#hide
df = (
    xr.DataArray(
        bar,
        coords=(
            foo.image_data["filt_C1_intensity_clones_and_neighbour_counts"].coords[
                "extended_labels_neighbour_counts"
            ],
            foo.image_data["filt_C1_intensity_clones_and_neighbour_counts"].coords[
                "img_name"
            ],
            range(bar.shape[2]),
        ),
        dims=("extended_labels_neighbour_counts", "img_name", "labels"),
    )
    .to_dataframe("co_label")
    .reset_index()
    .dropna()
)

In [ ]:
#hide
df.pivot(
    index=["img_name", "labels"],
    columns=["extended_labels_neighbour_counts"],
    values="co_label",
).query('C0_extended != 0').astype(np.uint16)

extended_labels_neighbour_counts   C0  C0_extended  C0_inside_clones  \
img_name labels                                                        
a1g01p1  11                        12           12                 0   
         12                        13           13                 0   
         13                        14           14                 0   
         14                        15           15                 0   
         15                        16           16                 0   
...                               ...          ...               ...   
a2g13p3  238                      239          239                 0   
         239                      240          240                 0   
         241                      242          242                 0   
         243                      244          244                 0   
         244                      245          245                 0   

extended_labels_neighbour_counts  C0_inside_clones_neighbour_counts  \
img_name labels                                                       
a1g01p1  11                                                       0   
         12                                                       0   
         13                                                       0   
         14                                                       0   
         15                                                       0   
...                                                             ...   
a2g13p3  238                                                      0   
         239                                                      0   
         241                                                      0   
         243                                                      0   
         244                                                      0   

extended_labels_neighbour_counts  C0_neighbour_counts  C0_outside_clones  \
img_name labels                                                            
a1g01p1  11                                         3                 12   
         12                                         3                 13   
         13                                         3                 14   
         14                                         5                 15   
         15                                         3                 16   
...                                               ...                ...   
a2g13p3  238                                        3                239   
         239                                        3                240   
         241                                        5                242   
         243                                        3                244   
         244                                        2                245   

extended_labels_neighbour_counts  C0_outside_clones_neighbour_counts  \
img_name labels                                                        
a1g01p1  11                                                        3   
         12                                                        2   
         13                                                        3   
         14                                                        5   
         15                                                        3   
...                                                              ...   
a2g13p3  238                                                       3   
         239                                                       3   
         241                                                       5   
         243                                                       3   
         244                                                       2   

extended_labels_neighbour_counts  merged_clones  
img_name labels                                  
a1g01p1  11                                   0  
         12                                   0  
         13  

In [ ]:
# hide
import napari

view = napari.Viewer()
foo.image_data
view.add_labels(foo.image_data["filt_C2_intensity_clones_and_neighbour_counts"].data)